# Economic Experiment

Add description of the experiment here

## Import Statements

In [1]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.accuracy import Likelihood
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.evaluators.base_evaluators import BettingActivity
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError

import networkx as nx
import pandas as pd
import numpy as np
import time

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## External actors

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [2]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)

In [3]:
betting = FixedBetting(100)

We also create an error rating that we are going to use for the calculated forecasts

In [4]:
rating_error = RatingFunctionError(error='normal', loc=0, scale=50)
rating_error_bookmaker = RatingFunctionError(error='normal', loc=0, scale=25)

## Generation of networks

Configure the number of leagues for the experiment:

In [5]:
numberLeagues = 20

For every league, we generate the schedule of matches and true dimensions and we add odds for each game.

In [6]:
networks = []
gen_start_time = time.time()
for league in range(numberLeagues):
    print(f"League : {league}")
    network: BaseNetwork = factory.new_network(
        'multiple-round-robin',
        teams=20,
        days_between_rounds=7,
        seasons=10,
        league_teams=20,
        league_promotion=0,
        create=True,
        true_forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients = [-0.9,0.3], 
            beta_parameter=0.006
        ),
        true_rating=ControlledTrendRating(
            starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
            delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
            trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
            season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)
        )
    )
    #adding biased bookmaker forecast to the network
    network.add_forecast(
        forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients=[-1.05, 0.15], 
            beta_parameter=0.006, 
            home_team_error=rating_error_bookmaker, 
            away_team_error=rating_error_bookmaker
        ),
        forecast_name='bookmaker_forecast',
        base_ranking='true_rating'
    )
    #adding odds    
    network.add_odds(
        bookmaker_name="bm",
        bookmaker=bookmaker,
        base_forecast='bookmaker_forecast'
    )

    networks.append(network)
print(f"{numberLeagues} leagues added in {float(time.time() - gen_start_time)} seconds.")

League : 0
Season 0
Relegation candidates {11: 11.0, 7: 13.0, 3: 14.0, 19: 18.0, 10: 18.0, 9: 19.0}
Relegation teams []
Season 1
Relegation candidates {0: 8.0, 11: 8.0, 6: 17.0, 8: 18.0, 16: 18.0, 10: 20.0}
Relegation teams []
Season 2
Relegation candidates {11: 9.0, 10: 10.0, 19: 16.0, 16: 17.0, 4: 17.0, 3: 17.0}
Relegation teams []
Season 3
Relegation candidates {4: 8.0, 11: 10.0, 10: 12.0, 3: 12.0, 14: 14.0, 8: 15.0}
Relegation teams []
Season 4
Relegation candidates {2: 11.0, 10: 13.0, 11: 13.0, 17: 17.0, 16: 18.0, 1: 21.0}
Relegation teams []
Season 5
Relegation candidates {16: 11.0, 0: 13.0, 11: 13.0, 14: 14.0, 3: 16.0, 4: 16.0}
Relegation teams []
Season 6
Relegation candidates {11: 9.0, 3: 10.0, 0: 13.0, 17: 14.0, 16: 16.0, 10: 17.0}
Relegation teams []
Season 7
Relegation candidates {11: 6.0, 10: 8.0, 3: 15.0, 19: 16.0, 16: 17.0, 0: 17.0}
Relegation teams []
Season 8
Relegation candidates {3: 9.0, 0: 10.0, 14: 10.0, 17: 11.0, 16: 17.0, 10: 17.0}
Relegation teams []
Season 9
Re

## Main loop

We iterate over the grid of possible parameters for the experiment and aggregate the final results.

In [7]:
def aggregate_measures(list_of_matches, prefix=''):
    return {prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]), 
           prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]), 
           prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
           prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches]),
           prefix+'_betting_activity': sum([m[3]['metrics']['betting_activity']['qty'] for m in list_of_matches])}

In [8]:
result_list = []
experiment_start_time = time.time()
for c0 in np.arange(-1.05, 0.0, 0.15):
    for c1 in np.arange(c0+0.3, 0.9, 0.15):
        for beta in np.arange(0.004, 0.010, 0.002):
            
            print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
            cell_start_time = time.time()
            all_matches =  []
            is_matches =  []
            oos_matches =  []
            
            for network in networks:
                rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
                betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
                betting_activity = BettingActivity(outcomes=['home', 'draw', 'away'], player_name = 'b')
                likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            
                network.add_forecast(
                forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
                forecast_name='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta),
                base_ranking='true_rating'
                )
                
                network.add_bets(
                bettor_name='b',
                bookmaker='bm',
                betting=betting,
                base_forecast='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)
                )
            
                network.add_evaluation(rps, 'rps')
                network.add_evaluation(betting_returns, 'betting_returns')
                network.add_evaluation(betting_activity, 'betting_activity')
                network.add_evaluation(likelihood, 'likelihood')
                #network.serialize_network('economic_network')
                
                all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
                is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 4]
                oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 3]
                
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
            }
            #print(len(all_matches))
            result_list.append(result)
            print(f"Finished in {float(time.time() - cell_start_time)} seconds")
            
                 
            
#network.export(forecasts = ['bookmaker_forecast', 'true_forecast', 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)],
#                      metrics = ['rps', 'betting_returns', 'loglikelihood'],
#                      odds=['bm'])
            
            
print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")
            

Variables: c0: -1.05, c1: -0.75 and beta: 0.004
Finished in 1.5429704189300537 seconds
Variables: c0: -1.05, c1: -0.75 and beta: 0.006
Finished in 1.3445303440093994 seconds
Variables: c0: -1.05, c1: -0.75 and beta: 0.008
Finished in 1.42816162109375 seconds
Variables: c0: -1.05, c1: -0.6 and beta: 0.004
Finished in 1.4193212985992432 seconds
Variables: c0: -1.05, c1: -0.6 and beta: 0.006
Finished in 1.30607271194458 seconds
Variables: c0: -1.05, c1: -0.6 and beta: 0.008
Finished in 1.4604225158691406 seconds
Variables: c0: -1.05, c1: -0.44999999999999996 and beta: 0.004
Finished in 1.3284432888031006 seconds
Variables: c0: -1.05, c1: -0.44999999999999996 and beta: 0.006
Finished in 1.5078856945037842 seconds
Variables: c0: -1.05, c1: -0.44999999999999996 and beta: 0.008
Finished in 1.384958267211914 seconds
Variables: c0: -1.05, c1: -0.29999999999999993 and beta: 0.004
Finished in 1.621384620666504 seconds
Variables: c0: -1.05, c1: -0.29999999999999993 and beta: 0.006
Finished in 1.77

In [9]:
df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")

In [ ]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])